In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from datetime import datetime, timedelta
from pandas import DataFrame, to_datetime, set_option
from TpsParse.Tps.TpsFile import TpsFile
import os
from numpy import quantile, nan
from math import ceil
from math import pi
import pandas as pd

set_option('display.max_columns', 500)
set_option('display.max_row', 2000)

from bokeh.resources import CDN
from bokeh.embed import components
from bokeh.models import Range1d, NumeralTickFormatter, ColumnDataSource, FactorRange, HoverTool
from bokeh.plotting import figure
from bokeh.transform import linear_cmap, factor_cmap, cumsum
from bokeh.io import show, output_notebook
from bokeh.palettes import Category20_20

from django.db.models import Sum, DateField
from django.db.models.functions import Trunc
from Utils.clean_string import clean_string

output_notebook()

Loading BokehJS ...

In [2]:
from app.models import Ventas, Items, Movimpu, Imputaci, Movgan, MovSilos, MovProd, Cajas, Gf, Giros, \
Cobrados, CtaCte, MovCons, MovLeche, Pagados, VctaCte, GanCar, Acumula, Actividad

In [3]:
# Obtener código de imputaciones
df2 = DataFrame(Imputaci.objects.values('codigo'))
df2.codigo.tolist()

[35,
 36,
 37,
 38,
 39,
 123,
 124,
 125,
 126,
 127,
 566,
 841,
 842,
 843,
 273,
 274,
 275,
 276,
 277,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 32,
 33,
 34,
 844,
 845,
 846,
 847,
 848,
 849,
 850,
 851,
 80,
 81,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 1325,
 1326,
 1327,
 1328,
 1329,
 1330,
 1331,
 1332,
 1333,
 1334,
 1335,
 1336,
 505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 515,
 335,
 336,
 337,
 338,
 339,
 340,
 341,
 342,
 343,
 344,
 345,
 346,
 347,
 363,
 364,
 365,
 366,
 367,
 368,
 369,
 370,
 371,
 372,
 373,
 374,
 388,
 389,
 390,
 391,
 392,
 393,
 394,
 395,
 396,
 397,
 398,
 399,
 400,
 736,
 737,
 738,
 739,
 740,
 741,
 742,
 743,
 744,
 745,
 746,
 747,
 748,
 749,
 750,
 751,
 752,
 753,
 754,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 1186,
 1189,
 1193,
 1194,
 1195,
 299,
 300,
 301,
 302,
 303,


In [4]:
# Obtener los códigos de imputación que no existen.
df1 = DataFrame(Movimpu.objects.values('cod_imput').distinct('cod_imput'))
df1.dropna(inplace=True)
df1 = df1.astype({'cod_imput': int})
df1[~df1.cod_imput.isin(df2.codigo.tolist())].cod_imput.tolist()

[]

In [5]:
df = DataFrame(Acumula.objects.values())
df.dropna(subset=['descripcion'], inplace=True)
df.descripcion

0                          Cadenas
1                     Extractotres
2                Sistema Aireación
3              Gastos Banco Mutual
4               Compra Maquinarias
5                          Sanidad
6                         Quinchos
7                          Aguadas
8                      Inoculantes
9                          Secadas
10                           Pagos
11                       Cobranzas
12                 Reposicion Caja
13                        Facturas
14                         Balanza
15                       Fungicida
16                         Tanques
17                      Comisiones
18                       Subsidios
19                          Cursos
20                         Pellets
21                         Tambo 2
22                       Movilidad
23                Compra Camioneta
24                            Moto
25                      Descuentos
26            Diferencia De Cambio
27            Pintura Herramientas
28                  

In [6]:
df = DataFrame(Actividad.objects.values())
df

,nro_actividad,descripcion
0,1,Mixta
1,2,Tambo
2,3,Agricultura


In [7]:
DataFrame(DataFrame(Movimpu.objects.values('tipo_movim', 'cod_imput__descripcion', 'actividad')).groupby(['tipo_movim']))

,0,1
0,Cobranza,tipo_movim cod_imput__descripci...
1,Documento De Contado,tipo_movim cod_imput__de...
2,Egreso,tipo_movim cod_imput__descripcion activ...
3,Facturas Cuenta Corriente,tipo_movim cod_imput__d...
4,Facturas De Contado,tipo_movim cod_imput__desc...
5,Ingreso,tipo_movim cod_imput__descripcion activ...
6,Nota De Crédito,tipo_movim cod_imput__descripcion...
7,Nota De Crédito,tipo_movim cod_imput__des...
8,Nota De Crédito Emitida,tipo_movim cod_imput__des...
9,Nota De Débito,tipo_movim cod_imput__desc...


In [8]:
acumuladores_ingreso = [
    'Cobranza', 'Ingreso', 'Ventas Cuenta Corriente', 'Ventas De Contado', 'Nota De Crédito', 'Nota De Débito Emitida'
]
acumuladores_egreso = [
    'Documento De Contado', 'Egreso', 'Facturas De Contado', 'Facturas Cuenta Corriente', 'Orden De Pago', 'Nota De Crédito Emitida', 'Nota De Débito'
]

In [9]:
movimpu_df = DataFrame(Movimpu.objects.values(
        'cod_imput__descripcion',
        'cod_imput__donde_acumula__descripcion',
        'actividad__descripcion', 'importe',
        'tipo_movim'))
movimpu_df.rename(columns={
    'cod_imput__descripcion': 'imputacion',
    'cod_imput__donde_acumula__descripcion': 'acumulador',
}, inplace=True); movimpu_df

,imputacion,acumulador,actividad__descripcion,importe,tipo_movim
0,None,None,None,12759.60,Egreso
1,Cosechadora Sts 9650,Maquinarias,None,0.00,Ventas De Contado
2,Cosechadora Sts 9650,Maquinarias,None,0.00,Ventas De Contado
3,Cosechadora Sts 9650,Maquinarias,None,0.00,Ventas De Contado
4,Cosechadora Sts 9650,Maquinarias,None,0.00,Ventas De Contado
...,...,...,...,...,...
56549,Ferretería,Ferretería,None,6734.40,Facturas Cuenta Corriente
56550,Ferretería,Ferretería,None,7276.33,Facturas Cuenta Corriente
56551,Pago Proveedores,None,None,16952.98,Orden De Pago
56552,Pago Proveedores,None,None,38789.56,Orden De Pago


In [10]:
movimpu_df.loc[movimpu_df.tipo_movim.isin(acumuladores_ingreso), 'tipo'] = 'Ingreso'
movimpu_df.loc[movimpu_df.tipo_movim.isin(acumuladores_egreso), 'tipo'] = 'Egreso'
movimpu_df.loc[movimpu_df.acumulador == 'Ventas', 'tipo'] = 'Ingreso'
movimpu_df.dropna(subset=['tipo'], inplace=True)
movimpu_df

,imputacion,acumulador,actividad__descripcion,importe,tipo_movim,tipo
0,None,None,None,12759.60,Egreso,Egreso
1,Cosechadora Sts 9650,Maquinarias,None,0.00,Ventas De Contado,Ingreso
2,Cosechadora Sts 9650,Maquinarias,None,0.00,Ventas De Contado,Ingreso
3,Cosechadora Sts 9650,Maquinarias,None,0.00,Ventas De Contado,Ingreso
4,Cosechadora Sts 9650,Maquinarias,None,0.00,Ventas De Contado,Ingreso
...,...,...,...,...,...,...
56549,Ferretería,Ferretería,None,6734.40,Facturas Cuenta Corriente,Egreso
56550,Ferretería,Ferretería,None,7276.33,Facturas Cuenta Corriente,Egreso
56551,Pago Proveedores,None,None,16952.98,Orden De Pago,Egreso
56552,Pago Proveedores,None,None,38789.56,Orden De Pago,Egreso


In [11]:
DataFrame(movimpu_df.groupby(['actividad__descripcion','tipo', 'acumulador']).sum())

importe
actividad__descripcion tipo    acumulador                 
Agricultura            Ingreso Hacienda           20986.13
                               Maquinarias            0.00
                               Ventas          12410315.60
Mixta                  Ingreso Fletes                 0.00
                               Maquinarias            0.00
Tambo                  Egreso  Alimento Vacas      1467.40
                               Sueldos             1483.18
                       Ingreso Maquinarias            0.00

In [153]:
venta_ganado = ['Ventas Animales', 'Venta De Hacienda I']
venta_tambo = ['Venta De Leche']
venta_agriculura = ['Ventas De Cereales']

impu_ganado = ['Crianza I', 'Flete Ganaderia', 'Gastos Ganaderia', 'Gastos Veterinaria', 
                       'Suplementación I', 'Suplementación Recría', 'Compra Hacienda']

impu_agriculura = ['Agroquimicos', 'Combustible', 'Fertilizantes', 'Flete Agricultura', 
                       'Gastos Varios Maquinaria', 'Inoculantes', 'Insumos De Semillas',
                      'Lubricante Maquinaria', 'Mixer', 'Personal Maquinaria', 'Reparación Tractores',
                      'Semillas', 'Silos', 'Rollos', 'Gastos De Forraje']

impu_tambo = ['Control Lechero', 'Energía Tambo', 'Higiene Tambo', 'Higiene Tambo I', 
                    'Honorarios Tambero', 'Ordeñadora I', 'Otros Gastos Tambo', 'Sanidad Tambo',
                   'Suplementación Tambo', 'Suplementación Guachera']

imputaciones = ['Crianza I', 'Flete Ganaderia', 'Gastos Ganaderia', 'Gastos Veterinaria', 
                       'Suplementación I', 'Suplementación Recría', 'Compra Hacienda', 'Agroquimicos', 'Combustible', 'Fertilizantes', 'Flete Agricultura', 
                       'Gastos Varios Maquinaria', 'Inoculantes', 'Insumos De Semillas',
                      'Lubricante Maquinaria', 'Mixer', 'Personal Maquinaria', 'Reparación Tractores',
                      'Semillas', 'Silos', 'Rollos', 'Gastos De Forraje', 'Control Lechero', 'Energía Tambo', 'Higiene Tambo', 'Higiene Tambo I', 
                    'Honorarios Tambero', 'Ordeñadora I', 'Otros Gastos Tambo', 'Sanidad Tambo',
                   'Suplementación Tambo', 'Suplementación Guachera']
imputacion_ventas = ['Ventas Animales', 'Venta De Hacienda I', 'Venta De Leche', 'Ventas De Cereales']

imputacion_ganado = ['Crianza I', 'Flete Ganaderia', 'Gastos Ganaderia', 'Gastos Veterinaria', 
               'Suplementación I', 'Suplementación Recría', 'Compra Hacienda', 
               'Ventas Animales', 'Venta De Hacienda I']

imputacion_agriculura = ['Agroquimicos', 'Combustible', 'Fertilizantes', 'Flete Agricultura', 
                   'Gastos Varios Maquinaria', 'Inoculantes', 'Insumos De Semillas',
                   'Lubricante Maquinaria', 'Mixer', 'Personal Maquinaria', 'Reparación Tractores',
                   'Semillas', 'Silos', 'Rollos', 'Gastos De Forraje', 'Ventas De Cereales']

imputacion_tambo = ['Control Lechero', 'Energía Tambo', 'Higiene Tambo', 'Higiene Tambo I', 'Honorarios Tambero', 
              'Ordeñadora I', 'Otros Gastos Tambo', 'Sanidad Tambo', 'Suplementación Tambo', 
              'Suplementación Guachera', 'Venta De Leche']

In [66]:
movimpu.loc[movimpu.imputacion_acumulada.isin(imputaciones), 'tipo'] = 'Costos'
movimpu.loc[movimpu.imputacion_acumulada.isin(impu_ventas), 'tipo'] = 'Ventas'

ganado = movimpu.loc[movimpu.imputacion_acumulada
    .isin(impu_ganado)].copy()
agricultura = movimpu.loc[movimpu.imputacion_acumulada
    .isin(impu_agriculura)].copy()
tambo = movimpu.loc[movimpu.imputacion_acumulada
    .isin(impu_tambo)].copy()

rentabilidad = {
    'ganado': ganado,
    'agricultura': agricultura,
    'tambo': tambo,
}

In [39]:
gb = DataFrame(movimpu.groupby('cod_imput__donde_acumula__descripcion').importe.sum()); gb

,importe
cod_imput__donde_acumula__descripcion,
Agroquimicos,2068319.19
Ajustes,109759.50
Arrendamientos,1044376.50
Asesor Contable,234250.00
Asesor Tecnico,16000.00
Bonificaciones,55245.06
Cobranzas,57101472.36
Combustible,658631.12
Comite De Cuenca,148808.13


In [41]:
costos = movimpu[movimpu.cod_imput__donde_acumula__descripcion.isin(imputacion_animales)].importe.sum()

In [42]:
ventas = movimpu[movimpu.cod_imput__donde_acumula__descripcion.isin(
    venta_animales)].importe.sum(); ventas

9085131.680000002

##### Filtrar los datos del año 2017

In [36]:
df = df[(df.periodo_contable >= '01/01/2017') & (df.periodo_contable <= '12/31/2017')]; df

,imputacion_simple,imputacion_acumulada,periodo_contable,tipo_movim,importe,actividad,tipo
8511,Aceites,Combustible,2017-01-01,Facturas De Contado,667.77,None,Egreso
8512,Vacunacion,Insumos Ganaderia,2017-01-01,Facturas De Contado,237.00,None,Egreso
8513,Nafta,Combustible,2017-01-01,Facturas De Contado,494.70,None,Egreso
8514,Gas,Combustible,2017-01-01,Facturas De Contado,156.20,None,Egreso
8515,Repuestos Y Reparaciones,Mantenimiento,2017-01-01,Facturas De Contado,850.00,None,Egreso
...,...,...,...,...,...,...,...
9587,Seguros Accidentes Personales,Seguros,2017-06-30,Facturas Cuenta Corriente,28.00,None,Egreso
9588,Intereses Creditos,Creditos,2017-06-30,Facturas Cuenta Corriente,5448.19,None,Egreso
9589,Intereses Creditos,Creditos,2017-06-30,Facturas Cuenta Corriente,8950.11,None,Egreso
9590,Repuestos Y Reparaciones,Mantenimiento,2017-06-30,Facturas Cuenta Corriente,6244.35,None,Egreso


Movimientos de Combustible

In [3]:
mov_cons = DataFrame(MovCons.objects.values('fecha', 'nro_comprob', 'ing_egr', 'costo')); mov_cons

,fecha,nro_comprob,ing_egr,costo
0,2020-03-09,1,1,1.0
1,2020-03-09,12,1,1.0
2,2020-03-09,111,2,451.0
3,2020-03-09,11,2,1.0


In [4]:
gb = DataFrame(mov_cons.groupby(['ing_egr']).costo.sum()); gb

,costo
ing_egr,
1,2.0
2,452.0


Movimientos de Caja

In [5]:
gf = DataFrame(Gf.objects.values('n_comp', 'fecha', 'i_o_e', 'importe')); gf

,n_comp,fecha,i_o_e,importe
0,11708,2018-06-20,2,90000.0
1,11709,2018-06-20,2,90000.0
2,11710,2018-07-20,2,90000.0
3,11711,2018-07-20,2,90000.0
4,11712,2018-08-20,2,90000.0
5,11713,2018-08-20,2,90000.0
6,75576767,2018-09-29,1,50000.0
7,75576768,2018-09-29,1,25000.0
8,75576769,2018-09-29,1,25000.0
9,72019,2019-08-02,2,150000.0


In [6]:
gb = DataFrame(gf.groupby(['i_o_e']).importe.sum()); gb

,importe
i_o_e,
1,100010.0
2,690000.0


Movimientos de Productos

In [7]:
items = DataFrame(Items.objects.values('fecha','nro_comprobante', 'movimiento', 
                                       'nro_producto__nro_linea__nombre')); items

,fecha,nro_comprobante,movimiento,nro_producto__nro_linea__nombre
0,2019-06-26,89766,1,Semilla
1,2020-03-09,12,1,Semilla
2,2019-06-26,89766,1,Silobolsa
3,2020-03-09,111,1,Silobolsa
4,2020-03-09,1,1,Silobolsa
5,2019-06-26,89766,1,None


In [8]:
gb = DataFrame(items.groupby(['movimiento'])); gb

,0,1
0,1,fecha nro_comprobante movimiento nro...


Compra y Venta de Ganado

In [9]:
mov_gan = DataFrame(Movgan.objects.values('nro_novedad', 'fecha', 'entrada_salida', 'categoria__descripcion', 'total')); mov_gan

,nro_novedad,fecha,entrada_salida,categoria__descripcion,total
0,337,2017-08-24,2,Vacas,8977.00
1,337,2017-08-24,2,Vacas,13038.00
2,421,2017-10-24,2,Vacas,14190.00
3,421,2017-10-24,2,Vacas,31244.00
4,421,2017-10-24,2,Vacas,9680.00
5,421,2017-10-24,2,Vacas,33908.00
6,421,2017-10-24,2,Vacas,7567.00
7,481,2017-10-24,2,Vacas,14190.00
8,481,2017-10-24,2,Vacas,31244.00
9,481,2017-10-24,2,Vacas,9680.00


In [10]:
gb = DataFrame(mov_gan.groupby('entrada_salida').total.sum()); gb

,total
entrada_salida,
1,1811.00
2,8420942.48


Movimientos de silos

In [11]:
mov_silos = DataFrame(MovSilos.objects.values('movimiento', 'fecha', 
                                              'nro_comprobante')); mov_silos

,movimiento,fecha,nro_comprobante
0,2,2019-05-15,12778688
1,2,2019-12-26,1255
2,2,2019-12-27,12778688
3,1,2020-03-09,0
4,1,2020-03-09,0
5,1,2020-03-09,0
6,2,2020-03-09,1
7,1,2020-03-09,1


In [12]:
gb = DataFrame(mov_silos.groupby(['movimiento'])); gb

,0,1
0,1,movimiento fecha nro_comprobante 3 ...
1,2,movimiento fecha nro_comprobante 0 ...


Movimientos de leche 

In [13]:
mov_prod = DataFrame(MovProd.objects.values('fecha', 'ing_egr', 'nro_comprob')); mov_prod

,fecha,ing_egr,nro_comprob
0,2019-07-01,2,0
1,2019-07-02,2,0
2,2020-03-09,2,1


In [14]:
gb = DataFrame(mov_prod.groupby(['ing_egr'])); gb

,0,1
0,2,fecha ing_egr nro_comprob 0 2019-07...


In [15]:
ventas = DataFrame(Ventas.objects.values('fecha_comprob', 'nro_compro', 'imp_total', 'tipo_movim__descripcion')); ventas

,fecha_comprob,nro_compro,imp_total,tipo_movim__descripcion
0,2016-10-17,16,613337.80,Orden de Pago
1,2016-12-17,17,312984.80,Orden de Pago
2,2016-12-17,18,334408.03,Orden de Pago
3,2016-12-17,19,500000.00,Orden de Pago
4,2016-12-17,20,149863.12,Orden de Pago
5,2016-10-31,1605,697833.40,Factura Cuenta Corriente
6,2016-10-24,9355,47672.07,Factura Cuenta Corriente
7,2016-11-30,18,711796.80,Factura Cuenta Corriente
8,2016-11-15,9436,286068.10,Factura Cuenta Corriente
9,2017-03-24,29,481603.20,Orden de Pago


In [24]:
costos = movimpu_df[movimpu_df.imputacion_acumulada.isin(
    impu_ganado)].importe.sum()
ventas = movimpu_df[movimpu_df.imputacion_acumulada.isin(
    venta_ganado)].importe.sum()
utilidad_neta = ventas - costos
utilidad = utilidad_neta / ventas
ganado = {
    'ventas': ventas,
    'costos': costos,
    'utilidad_neta': utilidad_neta,
    'utilidad': utilidad,
}

costos = movimpu_df[movimpu_df.imputacion_acumulada.isin(
    impu_agriculura)].importe.sum()
ventas = movimpu_df[movimpu_df.imputacion_acumulada.isin(
    venta_agriculura)].importe.sum()
utilidad_neta = ventas - costos
utilidad = utilidad_neta / ventas
agricultura = {
    'ventas': ventas,
    'costos': costos,
    'utilidad_neta': utilidad_neta,
    'utilidad': utilidad,
}

costos = movimpu_df[movimpu_df.imputacion_acumulada.isin(
    impu_tambo)].importe.sum()
ventas = movimpu_df[movimpu_df.imputacion_acumulada.isin(
    venta_tambo)].importe.sum()
utilidad_neta = ventas - costos
utilidad = utilidad_neta / ventas
tambo = {
    'ventas': ventas,
    'costos': costos,
    'utilidad_neta': utilidad_neta,
    'utilidad': utilidad,
}

rentabilidad = {
    'categoria': ['Ganado', 'Agricultura', 'Tambo'],
    'utilidad': [ganado['utilidad'], agricultura['utilidad'], tambo['utilidad']]
}

df = DataFrame(rentabilidad)
df.loc[df.utilidad < 0, 'color'] = 'red'
df.loc[df.utilidad > 0, 'color'] = 'green'
percentage = abs(df.utilidad) / abs(df.utilidad).sum() * 100
df['percentage'] = round(percentage, 1)
df['angle'] = abs(df.utilidad) / abs(df.utilidad).sum() * 2 * pi
source = ColumnDataSource(df)
source.data['percentage'] = [f'{x}%' for x in
                             source.data['percentage']]

tooltips = [("Categoría", "@categoria"),
            ("Utilidad", "@utilidad{$0,0.00}"),
            ('% del total', '@percentage')]
hovertool = HoverTool(names=['wedge'], tooltips=tooltips)
plot = figure(
    tools=["tap,reset,save", hovertool],
    toolbar_location='above', height=300)
plot.wedge(x=0, y=0, radius=0.4,
           start_angle=cumsum('angle', include_zero=True),
           end_angle=cumsum('angle'),
           line_color='white', line_width=0.4, 
           fill_color='color',
           legend_field='categoria',
           source=source, name='wedge')
show(plot)

In [19]:
source.data

{'index': array([0, 1, 2]),
 'categoria': array(['Ganado', 'Agricultura', 'Tambo'], dtype=object),
 'importe': array([ 0.66884054, -0.38542257,  0.66276966]),
 'percentage': ['70.7%', '-40.7%', '70.0%'],
 'angle': array([ 4.44145424, -2.55940934,  4.40114041])}